In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [8]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [9]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
89,46,Male,103.0,Strong,Bangalore
10,75,Female,NaN,Mild,Delhi
42,27,Male,100.0,Mild,Delhi
7,20,Female,NaN,Strong,Mumbai
16,69,Female,103.0,Mild,Kolkata
...,...,...,...,...,...
22,71,Female,98.0,Strong,Kolkata
61,81,Female,98.0,Strong,Mumbai
66,51,Male,104.0,Mild,Kolkata
12,25,Female,99.0,Strong,Kolkata


## 1. without col transformer(aam zindagi)

In [13]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
# the empty values got replced by mean of all the values

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [14]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [17]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [18]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [20]:
# here at the end we concatenate the whole dataframes and put them in 1 df
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

### using column transformer(mentos zindagi)

In [24]:
from sklearn.compose import ColumnTransformer

In [26]:
# here the remainder cols can be dropped or can be just left passthrough
# transformers are passed in the tuples -> each tuple is passed 3 values -> 1. the name of the transformer
# 2. the transformer output and 3. list of columns on which the transformation has to be performed
transformer = ColumnTransformer(transformers=[
     ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [27]:
transformer.fit_transform(X_train).shape

(80, 7)

In [28]:
transformer.transform(X_test).shape

(20, 7)